In [2]:
import yfinance as yf
import pandas as pd
import pyodbc
import datetime

In [ ]:
# Connection parameters
server = 'localhost\\SQLEXPRESS'   # Server name (as configured during installation)
database = 'FinanceDB'             # Name of the database (must exist in SQL Server)
connection_string = f"""
DRIVER={{ODBC Driver 17 for SQL Server}};
SERVER={server};
DATABASE={database};
Trusted_Connection=yes;
"""

# Try connecting to the database
try:
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()
    print("✅ Successfully connected to the database")
except Exception as e:
    print("❌ Connection error:", e)


✅ Успешное подключение к базе данных


In [ ]:
# Download parameters
ticker = "AAPL"
start_date = "2020-01-01"
end_date = "2024-12-31"

# Download data from Yahoo Finance
df = yf.download(ticker, start=start_date, end=end_date)

# Reset index to move 'Date' from index to column
df.reset_index(inplace=True)

# Add a Ticker column to identify the asset
df["Ticker"] = ticker

# Preview and check how many rows were loaded
print("🔽 Rows loaded:", len(df))
df.head()


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

🔽 Загружено строк: 1257


Price,Date,Close,High,Low,Open,Volume,Ticker
Ticker,,AAPL,AAPL,AAPL,AAPL,AAPL,
0,2020-01-02,72.620842,72.681289,71.373218,71.627092,135480400,AAPL
1,2020-01-03,71.914810,72.676439,71.689950,71.847110,146322800,AAPL
2,2020-01-06,72.487854,72.526541,70.783256,71.034717,118387200,AAPL
3,2020-01-07,72.146942,72.753823,71.926915,72.497529,108872000,AAPL
4,2020-01-08,73.307518,73.609752,71.849540,71.849540,132079200,AAPL


In [ ]:
# Download parameters
ticker = "^GSPC"
start_date = "1900-01-01"  # Earliest possible date
end_date = None            # Up to the current date

# Download data from Yahoo Finance
df = yf.download(ticker, start=start_date)

# Flatten column names in case a multi-index is returned
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)

# Reset index to make 'Date' a regular column
df.reset_index(inplace=True)

# Add a Ticker column to identify the asset
df["Ticker"] = ticker

# Preview and check how many rows were loaded
print("🔽 Rows loaded:", len(df))
df.head()

[*********************100%***********************]  1 of 1 completed

🔽 Загружено строк: 24469


Price,Date,Close,High,Low,Open,Volume,Ticker
0,1927-12-30,17.660000,17.660000,17.660000,17.660000,0,^GSPC
1,1928-01-03,17.760000,17.760000,17.760000,17.760000,0,^GSPC
2,1928-01-04,17.719999,17.719999,17.719999,17.719999,0,^GSPC
3,1928-01-05,17.549999,17.549999,17.549999,17.549999,0,^GSPC
4,1928-01-06,17.660000,17.660000,17.660000,17.660000,0,^GSPC


In [ ]:
# Create a new table without the 'AdjClose' column
create_table_query = """
CREATE TABLE StockPrices (
    ID INT IDENTITY(1,1) PRIMARY KEY,
    [Date] DATE,
    [Open] FLOAT,
    [High] FLOAT,
    [Low] FLOAT,
    [Close] FLOAT,
    [Volume] BIGINT,
    [Ticker] VARCHAR(10)
);
"""

# Execute the query to create the table
cursor.execute(create_table_query)
conn.commit()

print("✅ New table 'StockPrices' has been created"

✅ Новая таблица StockPrices создана


In [ ]:
# SQL insert query for inserting data into the StockPrices table
insert_query = """
INSERT INTO StockPrices ([Date], [Open], [High], [Low], [Close], [Volume], [Ticker])
VALUES (?, ?, ?, ?, ?, ?, ?)
"""

# Loop through the DataFrame and insert each row into the database
for _, row in df.iterrows():
    cursor.execute(insert_query, (
        row["Date"], row["Open"], row["High"], row["Low"],
        row["Close"], row["Volume"], row["Ticker"]
    ))

# Commit the transaction to save changes
conn.commit()
print("✅ Data successfully inserted into the StockPrices table")

✅ Данные успешно сохранены в новую таблицу


In [ ]:
# SQL query to select all data from the StockPrices table ordered by date
select_query = "SELECT * FROM StockPrices ORDER BY [Date];"

# Execute the query and load the result into a DataFrame
df_sql = pd.read_sql(select_query, conn)

# Display confirmation and show the first few rows
print("✅ Data successfully loaded from the database:")
print(df_sql.head())

✅ Данные загружены из базы:
   ID        Date       Open       High        Low      Close  Volume Ticker
0   1  1927-12-30  17.660000  17.660000  17.660000  17.660000       0  ^GSPC
1   2  1928-01-03  17.760000  17.760000  17.760000  17.760000       0  ^GSPC
2   3  1928-01-04  17.719999  17.719999  17.719999  17.719999       0  ^GSPC
3   4  1928-01-05  17.549999  17.549999  17.549999  17.549999       0  ^GSPC
4   5  1928-01-06  17.660000  17.660000  17.660000  17.660000       0  ^GSPC


C:\Users\m-a-a\AppData\Local\Temp\ipykernel_4660\991572164.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = pd.read_sql(select_query, conn)


In [ ]:
# SQL query to drop the StockPrices table if it exists
drop_table_query = "IF OBJECT_ID('StockPrices', 'U') IS NOT NULL DROP TABLE StockPrices;"

# Execute the drop table query
cursor.execute(drop_table_query)
conn.commit()

print("🗑️ Table 'StockPrices' has been dropped (if it existed)")